In [1]:
import tkinter as tk
from tkinter import ttk
import socket
import threading
from Techniques.des_encry_decry import bin_to_hexa,encrypt,pad,decrypt
from Techniques.rc4 import rc4_encrypt,rc4_decrypt

In [2]:
class CryptoChatClient:
    def __init__(self, root):
        self.root = root
        self.root.title("Client")

        self.crypto_technique_var = tk.StringVar()

        # Create a dropdown menu for crypto techniques
        crypto_label = tk.Label(root, text="Crypto Technique:")
        crypto_label.grid(row=0, column=0, padx=10, pady=10, sticky="e")

        crypto_options = ["AES", "RSA", "DES", "RC-4", "ELGAMMAL"]  # Add more as needed
        crypto_dropdown = ttk.Combobox(root, textvariable=self.crypto_technique_var, values=crypto_options)
        crypto_dropdown.grid(row=0, column=1, padx=10, pady=10, sticky="w")
        crypto_dropdown.set(crypto_options[0])  # Set default value

        # Create a text box for messages
        message_label = tk.Label(root, text="Message:")
        message_label.grid(row=1, column=0, padx=10, pady=10, sticky="e")

        self.message_entry = tk.Entry(root, width=40)
        self.message_entry.grid(row=1, column=1, padx=10, pady=10, sticky="w")

        # Create a chat window
        self.chat_window = tk.Text(root, height=15, width=50, state=tk.DISABLED)
        self.chat_window.grid(row=2, column=0, columnspan=2, padx=10, pady=10)

        # Create a send button
        send_button = tk.Button(root, text="Send", command=self.send_message)
        send_button.grid(row=3, column=0, columnspan=2, pady=10)

        # Initialize the socket (without connecting)
        self.my_socket = socket.socket()

        # Start a thread to listen for incoming connections
        threading.Thread(target=self.connector_thread).start()

    def connector_thread(self):
        try:
            # Connect the socket when needed
            self.my_socket.connect(("localhost", 4345))
            print("Connected to server")

            # Start a thread to listen for incoming messages
            threading.Thread(target=self.receive_messages).start()

        except ConnectionError as e:
            print(f"Connection error: {e}")

    def receive_messages(self):
        while True:
            try:
                data = self.my_socket.recv(1024).decode('utf-8')
                
                if not data:
                    break

                crypto_technique, message = data.split(":", 1)     
                     
                # Perform crypto operations on the received message using the selected technique
                if crypto_technique == "DES":
                    # Add decryption logic here
                    decrypted_message = bin_to_hexa(decrypt(message,key= "0F1571C947D9E859"))
                    self.display_message(f"{crypto_technique}: {decrypted_message}")
                if crypto_technique == "RC-4":
                    key = "SecretKey"
                    decrypted =rc4_decrypt(key, message)
                    
                    self.display_message(f"{crypto_technique}: {decrypted}")

            except ConnectionResetError:
                print("Connection closed.")
                break

    def send_message(self):
        try:          
            message = self.message_entry.get()
            crypto_technique = self.crypto_technique_var.get()

            # Perform crypto operations on the message using the selected technique
            if crypto_technique == "DES":
                plain_text=pad(message)            
                key= "0E329232EA6D0D73"
                key=pad(key)
                cipher_text=bin_to_hexa(encrypt(plain_text,key))
                self.display_message(f"{crypto_technique}: {cipher_text}")
                
            
            if crypto_technique == "RC-4":
                key = "SecretKey"
                cipher_text = rc4_encrypt(key, message)
                self.display_message(f"{crypto_technique}: {cipher_text}")


            # Send the message to the client
            data = f"{crypto_technique}:{cipher_text}"
            msg = bytes(data, 'utf-8')
            self.my_socket.send(msg)

        except ConnectionError as e:
            print(f"Connection error: {e}")

    def display_message(self, message):
        self.chat_window.config(state=tk.NORMAL)
        self.chat_window.insert(tk.END, message + "\n")
        self.chat_window.config(state=tk.DISABLED)

if __name__ == "__main__":
    root = tk.Tk()
    app = CryptoChatClient(root)
    root.mainloop()

Connected to server
Encryption
Message after initial permutation:  2589649075E8FD8F
The converted 56bit key is:  01101000111111000100010010100001000100010011111010010110

Round: Left key part: Right key part: SubKey used:
01       75E8FD8F        56B0BD75     A6120B4D4C25
02       56B0BD75        C6A62C4E     A3DF829C7968
03       C6A62C4E        738538B8     C5E2634E162A
04       738538B8        F596506E     C09D79F0D40B
05       F596506E        600F7E8B     48772468A3C8
06       600F7E8B        887FBC6C     A26E4CC66544
07       887FBC6C        C11BFC09     9C38661E8103
08       C11BFC09        67117CF2     129AB83347C3
09       67117CF2        9616FE23     710DEAA3202B
10       9616FE23        18B3FA41     09F48B713191
11       18B3FA41        42415649     4BAB4D126A9C
12       42415649        0BAE3B9E     CE5D01D80B25
13       0BAE3B9E        99E9B723     1667789316A0
14       99E9B723        BAD22845     8C78D881D31D
15       BAD22845        3CF03C0F     2B1A74CA48D8
16       5A00